# Transactions

## Creates temp tables

In [ ]:
USE AdventureWorks2017

DROP TABLE IF EXISTS Temp.MyUserInfo
DROP TABLE IF EXISTS Temp.MyUserPhone
DROP TABLE IF EXISTS Temp.MyUserName

CREATE TABLE Temp.MyUserName
(
    Id int IDENTITY PRIMARY KEY,
    FirstName NVARCHAR(35),
    SecondName NVARCHAR(50)
)


CREATE TABLE Temp.MyUserPhone
(
    Id int IDENTITY PRIMARY KEY,
    UserId int FOREIGN KEY REFERENCES Temp.MyUserName(Id) UNIQUE,
    TelephoneNumber CHAR(12)
    CHECK (TelephoneNumber LIKE ('([0-9][0-9][0-9])[0-9][0-9][0-9][0-9][0-9][0-9][0-9]'))
)


CREATE TABLE Temp.MyUserInfo
(
    Id int IDENTITY PRIMARY KEY,
    UserId int FOREIGN KEY REFERENCES Temp.MyUserName(Id),
    BirthDate DATE
)




## Default transactions examples

In [ ]:
USE AdventureWorks2017
BEGIN TRANSACTION
    INSERT INTO Temp.MyUserName VALUES('Nikita', 'Ivanov')
COMMIT TRANSACTION

SELECT *
FROM Temp.MyUserName

In [ ]:
USE AdventureWorks2017

BEGIN TRANSACTION

    DECLARE @Id int;

    INSERT INTO Temp.MyUserName
    VALUES('Nona', 'Grishaeva')

    SET @Id = @@IDENTITY

    INSERT INTO Temp.MyUserPhone
    VALUES(@Id, '(098)1988954')

    INSERT INTO Temp.MyUserInfo
    VALUES(@Id, '19560923')
COMMIT TRANSACTION



## Example save point transaction

In [ ]:
USE AdventureWorks2017

BEGIN TRANSACTION

    DECLARE @Id INT;

    INSERT INTO Temp.MyUserName
    VALUES ('Grinich', 'Fredmort')

    SET @Id = @@IDENTITY

    SAVE TRANSACTION SavePoint

    INSERT INTO Temp.MyUserInfo
    VALUES(@Id, '19520321')


    INSERT INTO Temp.MyUserPhone
    VALUES(@Id, '(067)8660986')

COMMIT TRANSACTION

## Example with create procedure

In [ ]:
USE AdventureWorks2017
GO


DROP PROC IF EXISTS InsertUserData
GO
CREATE PROCEDURE InsertUserData 
    @UserFirstName NVARCHAR(25),
    @UserSecondName NVARCHAR(25),
    @UserPhone CHAR(12),
    @UserBirthDate DATE
AS
BEGIN
    DECLARE @Id INT;

    BEGIN TRANSACTION
        INSERT INTO Temp.MyUserName VALUES (@UserFirstName, @UserSecondName);
        SET @Id = @@IDENTITY
        INSERT INTO Temp.MyUserPhone VALUES (@Id, @UserPhone)
        INSERT INTO Temp.MyUserInfo VALUES (@Id, @UserBirthDate)

        IF EXISTS (SELECT Id FROM Temp.MyUserName WHERE FirstName = @UserFirstName AND SecondName = @UserSecondName AND Id != @Id )
            BEGIN ROLLBACK TRANSACTION
            RETURN 1
            END

    COMMIT TRANSACTION

END

In [25]:
USE AdventureWorks2017

EXEC InsertUserData @UserFirstName = 'Grinch', @UserSecondName = 'Geen', @UserPhone = '(055)9055854', @UserBirthDate = '19900812'

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.013

In [26]:
USE AdventureWorks2017


SELECT *
FROM Temp.MyUserName as N
LEFT JOIN Temp.MyUserPhone as P on P.UserId = N.Id
LEFT JOIN Temp.MyUserInfo as I on I.UserId = N.Id



-- TRUNCATE TABLE Temp.MyUserPhone
-- TRUNCATE TABLE Temp.MyUserInfo
-- TRUNCATE TABLE Temp.MyUserName


(3 rows affected)

Total execution time: 00:00:00.018

Id,FirstName,SecondName,Id,UserId,TelephoneNumber,Id,UserId,BirthDate
1,Nona,Grishaeva,NULL,NULL,NULL,NULL,NULL,NULL
5,Daemon,Tools,1,5,(095)9077854,1,5,1976-05-02
7,Grinch,Geen,3,7,(055)9055854,3,7,1990-08-12
